In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
SS_VER = 'SS_VER_2_1'
PATH = "/home/xjaros2/Documents/git/csidh-setup/csidh-target/src/"
attack_type = "A1"

In [2]:
from csidh import CSIDHCW
import chipwhisperer as cw
from tqdm import tqdm
import random

In [3]:
csidh = CSIDHCW(PATH, attack_type=attack_type)
csidh.setup()

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7370129.87012987         
scope.clock.adc_freq                     changed from 0                         to 29480519.48051948        
scope.clock.extclk_monitor_enabled       changed from True                      to False                    
scope.clock.extclk_tolerance             changed from 1144409.1796875           to 13096723.705530167       
scope.io.tio1                            changed from serial_tx                 to serial_rx         

In [4]:
csidh.flash_target()

SS_VER set to SS_VER_2_1
.
-------- begin --------
.
Cleaning project:
rm -f -- main-CW308_CC2538.hex main-CW301_AVR.hex main-CW303.hex main-CW304.hex main-CW308_MEGARF.hex main-CW308_SAM4L.hex main-CW308_STM32F0.hex main-CW308_STM32F1.hex main-CW308_STM32F2.hex main-CW308_STM32F3.hex main-CW308_STM32F4.hex main-CW308_K24F.hex main-CW308_NRF52.hex main-CW308_AURIX.hex main-CW308_SAML11.hex main-CW308_EFM32TG11B.hex main-CWLITEARM.hex main-CWLITEXMEGA.hex main-CWNANO.hex main-CW308_K82F.hex main-CW308_PSOC62.hex main-CW308_IMXRT1062.hex main-CW308_FE310.hex main-CW308_EFR32MG21A.hex main-CW308_EFM32GG11.hex main-CW308_STM32L5.hex main-CW308_NEORV32.hex main-CW308_SAM4S.hex main-CW305_IBEX.hex
rm -f -- main-CW308_CC2538.eep main-CW301_AVR.eep main-CW303.eep main-CW304.eep main-CW308_MEGARF.eep main-CW308_SAM4L.eep main-CW308_STM32F0.eep main-CW308_STM32F1.eep main-CW308_STM32F2.eep main-CW308_STM32F3.eep main-CW308_STM32F4.eep main-CW308_K24F.eep main-CW308_NRF52.eep main-CW308_AURIX.eep

In [29]:
csidh.reset_target()
csidh.scope.arm()
csidh.action()
ret = csidh.scope.capture()
if ret:
    print("Timeout happened during acquisition")
PUBLIC_EXPECTED = csidh.public_with_errors
max_ext_offset = csidh.scope.adc.trig_count
print("Public key:", PUBLIC_EXPECTED)
print("Max ext offset:", max_ext_offset)

Public key: 0
Max ext offset: 557892


In [6]:
EXT_OFFSET_FULL = 10528968
print(EXT_OFFSET_FULL - 10055348)
print(EXT_OFFSET_FULL - 10054576)
print(EXT_OFFSET_FULL - 10055640)

473620
474392
473328


In [7]:
# isogeny_offsets = [
#     (5,474252), # The if statement in both branches before isogenies cost 8 cycles  
#     (7,574920),
#     (5,1109332),
#     (3,1638652),
#     (5,1799384),
#     (7,1952772),
#     (3,2377220)
# ]
isogeny_offsets = [
    (5, 118555, 139478),
    (7, 161541, 175472),
    (5, 312859, 333855), 
    (3, 462833, 478086),
    (5, 508058, 529109),
    (7, 551170, 565075),
    (3, 672017, 687331),
    #(3, 717500, 738726)
]



In [8]:
csidh.voltage_glitching_setup()
# csidh.scope.cglitch_setup()

In [9]:
from csidh.search import Unit, generate_population, write_cache_to_file
from collections import OrderedDict
import numpy as np
cache = OrderedDict()


def evaluate_unit(csidh, unit, num_measurements=1):
    """Evaluates a single unit"""
    csidh.scope.glitch.num_glitches = 1
    if csidh.scope._is_husky:
        csidh.scope.glitch.width = int(unit.width)
        csidh.scope.glitch.offset = int(unit.offset)
        csidh.scope.glitch.repeat = int(unit.repeat)
        csidh.scope.glitch.ext_offset = int(unit.ext_offset)
    else:
        csidh.scope.glitch.width = unit.width
        csidh.scope.glitch.offset = unit.offset
        csidh.scope.glitch.repeat = unit.repeat
        csidh.scope.glitch.ext_offset = unit.ext_offset

    # Perform the measurements
    measurements = []
    responses = []

    for _ in range(num_measurements):
        csidh.reset_target()
        csidh.scope.glitch.state = None
        csidh.scope.arm()
        ret = csidh.action()
        csidh.scope.io.vglitch_reset()
        if ret:
            logging.error("Timeout happened during acquisition")


        public_received = csidh.public_with_errors
        if not isinstance(public_received, int):
            measurements.append("RESET")
        elif public_received == PUBLIC_EXPECTED:
            measurements.append("NORMAL")
        else:
            measurements.append("JUSTRIGHT")
            responses.append(public_received)

    unit.width = csidh.scope.glitch.width  # CW rounds the values
    unit.offset = csidh.scope.glitch.offset
    unit.repeat = csidh.scope.glitch.repeat
    unit.measurements = measurements
    unit.responses = responses
    

    # Classify
    if not all(m == measurements[0] for m in measurements):
        unit.type = "CHANGING"
        N_normal = sum(1 for m in measurements if m == "NORMAL")
        N_reset = sum(1 for m in measurements if m == "RESET")
        N_justright = sum(1 for m in measurements if m == "JUSTRIGHT")
        unit.fitness = 4 + 1.2 * N_justright + 0.2 * N_normal + 0.5 * N_reset
    else:
        if measurements[0] == "NORMAL":
            unit.type = "NORMAL"
            unit.fitness = 2
        elif measurements[0] == "RESET":
            unit.type = "RESET"
            unit.fitness = 5
        elif measurements[0] == "JUSTRIGHT":
            unit.type = "JUSTRIGHT"
            unit.fitness = 10
    cache[unit] = unit.fitness
    #if unit.type in ["RESET", "JUSTRIGHT"]:
    print(unit)
    print(unit.measurements)
    print(unit.responses)


def evaluate_batch(csidh, population):
    uncached = [u for u in population if u not in cache]
    if not uncached:
        return [u.fitness for u in population]

    to_visit = np.array(uncached)

    for unit in population:
        if unit in cache:
            unit.fitness = cache[unit]

    for unit in to_visit:
        evaluate_unit(csidh, unit)

    return [u.fitness for u in population]

In [10]:
csidh.scope.adc.timeout = 0.5

In [11]:
def generate_whole_population():
    population = []
    for offset in range(Unit.OFFSET_MIN, Unit.OFFSET_MAX+1):
        for width in range(Unit.WIDTH_MIN, Unit.WIDTH_MAX+1):
            for ext_offset in range(Unit.EXT_OFFSET_MIN, Unit.EXT_OFFSET_MAX+1):
                for repeat in range(Unit.REPEAT_MIN, Unit.REPEAT_MAX+1, 1):
                    unit = Unit(repr=f"{ext_offset},{offset},{width},{repeat},None,0")
                    population.append(unit)
    return population

In [12]:
from copy import deepcopy
import json
import pandas as pd
import seaborn as sns
from csidh.search import Unit

import os
os.chdir("/home/xjaros2/Documents/git/csidh-setup/csidh-target/scripts/")

def read_cachefile(filename):
    with open(filename, "r") as f:
        measurements = json.load(f)
    measurements = measurements["measurements"]
    result = []
    for i in range(len(measurements)):
        unit = Unit(repr=measurements[i]["unit"])
        measurements[i].update(unit.__dict__())
        unit.measurements = measurements[i]["measurements"]
        unit.responses = measurements[i]["responses"]
        del measurements[i]["index"]
        measurements[i]["unit"] = unit
        result.append(measurements[i])
    return result

def read_cachefiles_to_dataframe(cachefiles):
    df = None
    for filename in cachefiles:
        result = pd.DataFrame(read_cachefile(filename))
        if df is None:
            df = result
        else:
            df = pd.concat([df, result], ignore_index=True, sort=False)
    return df

In [13]:
husky = [
    "./husky-clock-xISOG-first-5-isogeny-skip-parameter-search.json"
]

df = read_cachefiles_to_dataframe(husky)
df = df[(df["type"] == "RESET") | (df["type"] == "JUSTRIGHT")]
candidates = df[((df["width"] < 750) & (df["offset"] < 1000)) | ((df["width"] < 750) & (df["offset"] > 3500))] 
candidates

,unit,responses,measurements,ext_offset,offset,width,repeat,type,fitness,offset_fine,width_fine
73,"(ext_offset=120679, offset=4431, width=274, re...",[],[RESET],120679,4431,274,11,RESET,5.0,None,None
76,"(ext_offset=121009, offset=4404, width=520, re...",[],[RESET],121009,4404,520,11,RESET,5.0,None,None
80,"(ext_offset=121351, offset=4305, width=434, re...",[],[RESET],121351,4305,434,8,RESET,5.0,None,None
148,"(ext_offset=120862, offset=37, width=385, repe...",[],[RESET],120862,37,385,15,RESET,5.0,None,None
283,"(ext_offset=121433, offset=4112, width=213, re...",[],[RESET],121433,4112,213,17,RESET,5.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...
15136,"(ext_offset=121120, offset=4361, width=508, re...",[],[RESET],121120,4361,508,4,RESET,5.0,None,None
15146,"(ext_offset=120836, offset=4302, width=490, re...",[],[RESET],120836,4302,490,14,RESET,5.0,None,None
15147,"(ext_offset=121223, offset=4352, width=325, re...",[],[RESET],121223,4352,325,8,RESET,5.0,None,None
15163,"(ext_offset=121450, offset=17, width=257, repe...",[],[RESET],121450,17,257,14,RESET,5.0,None,None


In [14]:
candidates = list(set(candidates["unit"]))

In [15]:
# potential = [
#     "./husky-clock-xISOG-first-5-isogeny-skip-search-until7.json"
# ]

# df = read_cachefiles_to_dataframe(potential)
# df = df[(df["type"] == "JUSTRIGHT")].reset_index()
# indices = [i  for i,x in enumerate(df["responses"]) if 199 in x]
# potential_skips = df.iloc[indices]
# batch = list(set(potential_skips["unit"]))
# batch.sort(key=lambda unit: -unit.ext_offset)
# batch
# cache = OrderedDict()
# uut = batch[-1]
# uut.offset = 2706
# uut.width = 2600
# uut.repeat = 4
# evaluate_batch(csidh, [uut])
# write_cache_to_file(f"first-5-isogeny-potential-skips.json", cache, 1, len(cache), -1)   

In [16]:
# cache = OrderedDict()
# REPEATS = 20000
# cache_interval = 2
# for _ in tqdm(range(REPEATS//cache_interval)):
#     batch = []
#     for _ in range(cache_interval):
#         i = random.randint(0, len(candidates)-1)
#         base_unit = candidates[i]
#         for j in range(len(isogeny_offsets)):
#             if j == 0:
#                 eof_min = 0
#                 eof_max = isogeny_offsets[0][1]
#             else:
#                 eof_min = isogeny_offsets[j-1][2]
#                 eof_max = isogeny_offsets[j][1]
#             unit = Unit(repr=repr(base_unit))
#             unit.ext_offset = random.randint(eof_min, eof_max)
#             print(unit)
#             batch.append(unit)

#     evaluate_batch(csidh, batch)
#     write_cache_to_file(f"husky-clock-ISOGENY-SKIP-SEARCH.json", cache, 1, len(cache), -1)   

In [17]:
# cache = OrderedDict()

# REPEATS = 17500
# cache_interval = 10
# for _ in tqdm(range(REPEATS//cache_interval)):
#     batch = []
#     for _ in range(cache_interval):
#         # width = random.randint(2650, 2750)
#         # offset = random.randint(2560, 2610)
#         # repeat = random.randint(4, 5)
#         width = random.randint(0, csidh.scope.glitch.phase_shift_steps//2)
#         offset = random.randint(0, csidh.scope.glitch.phase_shift_steps)
#         repeat = random.randint(1, 18)
        
#         ext_offset = random.randint((474252//4 )+2000, (474252//4 ) + 3200)
    
#         unit = Unit(repr=f"{ext_offset},{offset},{width},{repeat},None,0")
#         batch.append(unit)

#     evaluate_batch(csidh, batch)
#     write_cache_to_file(f"husky-clock-xISOG-first-5-isogeny-skip-parameter-search.json", cache, 1, len(cache), -1)   

In [ ]:
while True:
    csidh.scope.glitch.num_glitches = 2
    csidh.scope.glitch.width = 1600
    csidh.scope.glitch.offset = 2000
    csidh.scope.glitch.repeat = [1, 2]
    csidh.scope.glitch.ext_offset = [1, 5]

    
    csidh.reset_target()
    csidh.scope.glitch.state = None
    csidh.scope.arm()
    csidh.action()
    csidh.scope.io.vglitch_reset()
    ret = csidh.scope.capture()
    if ret:
        print("Timeout happened during acquisition")
    PUBLIC_EXPECTED = csidh.public_with_errors
    max_ext_offset = csidh.scope.adc.trig_count
    print("Public key:", PUBLIC_EXPECTED)
    print("Max ext offset:", max_ext_offset)